In [1]:
import pandas as pd
import json
import os
import numpy as np

import sentiwordnet
import wndomains
import geowordnet

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
#import spacy
import nltk
nltk.download('stopwords')

from nltk.corpus import wordnet as wn

# spacy for lemmatization
import spacy
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en')

C:\Users\TUSHAR\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TUSHAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# tokenize
def tokenize_words(text):
    token = []
    count=0
    lemmat_tokens=[]
    wordnet_lemmatizer = WordNetLemmatizer()
    total = len(text)
    #print((total))    
    try:        
        for i in range(total):
            count=count+1
            if(text[i] == '' or pd.isnull(text[i])):
                #print("empty str")
                num=9
            else:
                swtoken=[]
                swrtoken=[]
                swtoken=gensim.utils.simple_preprocess(text[i],deacc=True)
                token.append(swtoken)
                    
    except:
        print("text = ",text[i])
    
    return token 

In [3]:
path = 'C:\\Users\\TUSHAR\\Desktop\\unis_MSD\\capstone project\\notion\\dataset\\R4'
os.chdir(path)

data_df=pd.read_csv('article_content.csv', encoding='utf-8')
#data_df=pd.read_csv('20_article_content.csv', encoding='utf-8')
summary_text=list(data_df['content'])

from nltk.corpus import stopwords
stopWords = stopwords.words('english')
#stopWords = list(stopwords.words('english'))
stopWords.extend(["commission","royal","said","australia","let","told","also","ms","com","www","https","au","us","would","mailto","subject","body","says","orr","acbf","mr"]) 

#tokens = tokenize_words(summary_text)
data_words = tokenize_words(summary_text)

big_corpus=''
for article in summary_text:
    big_corpus=big_corpus+np.str(article)

In [8]:
def spacyNER(text):
    nlp = spacy.load('en')
    nerList=[]
    doc=nlp(text)
    for ent in doc.ents:
        nerEntry=ent.text+'_'+ent.label_
        nerList.append(nerEntry)
    print(nerList)
    return nerList

#big_corpus1='Apple is a big company in China'
#for article in summary_text[:10]:
    #big_corpus=big_corpus+np.str(article)

#nerList = spacyNER(big_corpus1)

In [5]:
#print(nerList)

In [4]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=2) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=2)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\TUSHAR\Anaconda3\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [5]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopWords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def spacyNER(text):
    nlp = spacy.load('en_core_web_sm')
    nerList=[]
    doc=nlp(text)
    for ent in doc.ents:
        nerEntry=ent.text+'_'+ent.label_
        nerList.append(nerEntry)
    return nerList

In [6]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_trigrams(data_words_nostops)

nlp = spacy.load('en')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#nerList = spacyNER(big_corpus)
#print(data_lemmatized[:1])

In [7]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [8]:
# Build LDA model
lmodel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           iterations=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [9]:
ltopics=[]
for i in range(lmodel.num_topics):
    topic_list_n=lmodel.print_topic(i)
    temp_token_1=topic_list_n.split('+')
    for ctoken in temp_token_1:
        temp_token_2=ctoken.split('*')
        temp_token_3 = temp_token_2[1]
        temp_token_4=temp_token_3.split('"')
        ind_topic=temp_token_4[1]
    #print(itop)
        ltopics.append(np.str(ind_topic))
    
print(ltopics)
unique_topic_list=list(set(ltopics))
print(unique_topic_list)

['investor', 'market', 'fall', 'stock', 'growth', 'price', 'tighten', 'today', 'sector', 'line', 'fund', 'manage', 'member', 'interest_rate', 'look', 'government', 'set', 'level', 'own', 'invest', 'funeral_plan', 'fund', 'community_benefit', 'fall', 'jone', 'funeral_insurer', 'corporate_watchdog', 'describe', 'advertising', 'scrutiny_aboriginal', 'bank', 'loan', 'banking', 'customer', 'policy', 'hear', 'call', 'sell', 'company', 'meet', 'think', 'need', 'year', 'australian', 'important', 'company', 'want', 'new', 'industry', 'china', 'funeral_insurance', 'walsh', 'family', 'work', 'indigenous_communitie', 'tracey_walsh', 'insurer', 'indigenous_consumer', 'fund', 'probe', 'bank', 'rate', 'borrower', 'continue', 'cost', 'ability', 'growth', 'increase', 'share', 'big_four', 'child', 'torres_strait', 'aboriginal', 'funeral', 'admit', 'rowena', 'targeted_indigenous', 'counsel_assisting', 'community', 'polici', 'monetary_policy', 'response', 'financial_crisis', 'limit', 'use', 'next_three', 

In [10]:
print(len(unique_topic_list))

262


In [11]:
interested_doomains1=['administration','banking','commerce','economy','finance','health','humanities','insurance','money','social_science','tax']
interested_doomains2=['geography','banking','economy','finance','insurance','social_science']
interested_doomains3=['geography','law','finance','social_science']
interested_doomains=['sociology','geography','law','social_science']
interested_doomains5=['sociology','social_science']
interested_doomains6=['banking','economy','finance']
interested_topics=[]
strength_list=[]

for composite_topic in unique_topic_list:
    try:
        tlist=composite_topic.split('_')
        for topic in tlist:
            s = wn.synsets(topic)[0]
            msc = wn.synset(s.name())
            dtup=msc.domains()[:5]
            #if composite_topic== 'funeral_insurance':
                #print("funeral_insurance", dtup)
            for tup in dtup:
                domain=tup[0]
                strength=tup[1]
                if domain in interested_doomains:
                    if (composite_topic not in interested_topics and s.pos() == 'n'):
                        interested_topics.append(composite_topic)
                        strength_list.append(strength)
                        #print('topic= ',composite_topic,'domain= ',domain,'strength= ','strength= ',strength, 'pos= ',s.pos())
    except:
        print('exception for topic ',composite_topic)
            
print(interested_topics)
print(len(interested_topics))

dfn=pd.DataFrame({'topic':interested_topics,'strength':strength_list})
#dfn.columns=['topic','strength']
len_total=dfn.topic.count()
len_desired=np.int(len_total - round(len_total*.1))
print('len_desired ',len_desired)

sdf = dfn.sort_values(['strength'], ascending=False)['topic']
#df_top=sdf['topic'].tolist()
temp_topics=sdf.iloc[:np.int(len_desired)]
final_topics=temp_topics.tolist()
#print(df_top.head())
print(final_topics)

exception for topic  jone
exception for topic  sales_representatif
exception for topic  tuesday_asic
exception for topic  walsh
exception for topic  torres_strait_islander_people
exception for topic  torres_strait
exception for topic  rba
exception for topic  australian_securitie
exception for topic  theaustralian
exception for topic  howden
exception for topic  lynda_edward
exception for topic  indigenous_communitie
exception for topic  tracey_walsh
exception for topic  polici
exception for topic  heard_anz
exception for topic  rowena
exception for topic  per_cent
exception for topic  ceo_bryn
exception for topic  smh
exception for topic  brauer
exception for topic  westpac
exception for topic  cba
exception for topic  asx_index
exception for topic  financial_plann
exception for topic  prime_minister_malcolm_turnbull
exception for topic  marika
exception for topic  victoria_believ
exception for topic  nathan_boyle
exception for topic  clayton_utz
['aboriginal', 'policy', 'cut', 'austr

In [12]:
# Test WNXnet
#s = wn.synsets('mortgage')[0]
#print(s.name())
#msc = wn.synset(s.name())
#print(msc.domains()[:5])
#dict1=msc.domains()[:5]
#for key in dict1:
    #domain=key[0]
    #print(domain, key[1])

In [13]:
topics1=['aboriginal','islander','indigenous','mortality','breaches']
#topics=['housing','loans','decreases','consecutive','trend']
#topics2=unique_topic_list
topics2=final_topics
topics1=final_topics

#tokensl1=tokens
len_docs=len(data_lemmatized)
print(len_docs)
window=50
topic_freq=dict()
co_occur_net=dict()

for t1 in topics1:
    for t2 in topics2:
        if(t1!= t2):
            #Loop through the docs
            for i in range(len_docs):
                tokensl1=data_lemmatized[i]                
                l=len(tokensl1)
                #Loop through the tokens of a document
                for j in range(l):
                    if(tokensl1[j]==t1):
                        if(j+1+window > l-1):
                            end=l
                        else:
                            end=j+1+window
            
                        str=tokensl1[j+1:end]
                        #print(str)
                        if(t2 in str):
                            #print(t1,t2,"co-occurrence")
                            tkey=t1+':'+t2
                            if(tkey in co_occur_net.keys()):
                                co_freq=co_occur_net[tkey]
                                co_occur_net[tkey]=co_freq+1
                            else:
                                co_occur_net[tkey]=1
                                

711


In [14]:
#Get Topic Frequency

topic_freq=dict()

for t1 in topics1:
    for i in range(len_docs):
        tokensl1=data_lemmatized[i]
        l=len(tokensl1)
        #Loop through the tokens of a document
        for j in range(l):
            if(tokensl1[j]==t1):
                if(t1 in topic_freq.keys()):
                    topic_freq[t1]=topic_freq[t1]+1
                else:
                    topic_freq[t1]=1

print(topic_freq)
print("co_occur")

tot_freq = 0
max_freq = 0
min_freq=9999
for topic in topic_freq.keys():
    tfreq= topic_freq[topic]
    if(tfreq > max_freq):
        max_freq=tfreq
    if(tfreq<min_freq):
        min_freq=tfreq
    tot_freq=tot_freq+tfreq

delta_freq=max_freq-min_freq
print(tot_freq)
print(delta_freq)
print((max_freq-min_freq)/tot_freq)

{'law_firm': 5, 'member': 44, 'group': 63, 'state': 29, 'sunshine_coast': 27, 'china': 38, 'people': 104, 'vulnerable_people': 6, 'indigenous_people': 41, 'economy': 50, 'death': 27, 'funeral': 32, 'costly_funeral': 4, 'funeral_insurer': 14, 'funeral_insurance': 112, 'funeral_plan': 16, 'report': 76, 'trade_war': 24, 'war': 35, 'time': 136, 'level': 39, 'counsel_assisting': 23, 'government': 81, 'home_loan': 45, 'interest': 52, 'interest_rate': 36, 'interest_mortgage': 4, 'indigenous_community': 6, 'community_benefit': 17, 'community': 53, 'failure': 19, 'policy': 105, 'monetary_policy': 26, 'back': 81, 'major_bank': 25, 'bank': 490, 'rural_bank': 30, 'reason': 19, 'world_show_subsections_world': 13, 'world': 32, 'see': 101, 'independent_report': 5, 'cut': 52, 'tax_cut': 19, 'due': 46, 'process': 36, 'claim': 28, 'brisbane': 25, 'sydney_call': 10, 'sydney': 41, 'wealth_management': 39, 'campaign': 27, 'response': 15, 'opportunity': 51, 'incentive_schemes': 10, 'wait': 11, 'aboriginal_p

In [15]:
# Edge Weighing as per Inverse Frequency evaluation
edge_weight_conet=dict()

for key,val in co_occur_net.items():
    composite_topics=key.split(':')
    source_topic=composite_topics[0]    
    if(source_topic in topic_freq.keys()):
        ind_freq=topic_freq[source_topic]
        if(ind_freq > 0):
            ew= val/np.int(ind_freq)
            if(ew>0.2):
                edge_weight_conet[key]=ew            

In [16]:
# Edge Weighing as per Inverse Frequency evaluation
print(edge_weight_conet)

{'law_firm:report': 0.4, 'member:australian': 0.20454545454545456, 'state:group': 0.27586206896551724, 'state:bank': 0.27586206896551724, 'state:see': 0.20689655172413793, 'sunshine_coast:funeral_insurance': 0.2222222222222222, 'sunshine_coast:sydney': 0.3333333333333333, 'sunshine_coast:incentive_schemes': 0.3333333333333333, 'china:world': 0.21052631578947367, 'people:time': 0.20192307692307693, 'vulnerable_people:people': 0.5, 'vulnerable_people:funeral': 0.5, 'vulnerable_people:funeral_insurance': 0.5, 'vulnerable_people:indigenous_community': 0.5, 'vulnerable_people:community_benefit': 0.3333333333333333, 'vulnerable_people:aboriginal_organisation': 0.3333333333333333, 'vulnerable_people:darwin_tuesday': 0.3333333333333333, 'indigenous_people:aboriginal': 0.2926829268292683, 'economy:politics_federal': 0.24, 'death:funeral_insurance': 0.2222222222222222, 'death:funeral_plan': 0.25925925925925924, 'death:see': 0.2962962962962963, 'death:due': 0.2962962962962963, 'death:probe': 0.25

In [20]:
# # Edge Weighing as per Direct Frequency evaluation
edge_weight_direct=dict()

for key,val in co_occur_net.items():
    composite_topics=key.split(':')
    source_topic=composite_topics[0]    
    if(source_topic in topic_freq.keys()):
        ind_freq=topic_freq[source_topic]
        if(ind_freq > 0):
            ew= (val*np.int(ind_freq))/delta_freq
            if(ew>2):
                edge_weight_direct[key]=ew
                
print(edge_weight_direct)

{'people:time': 4.484599589322382, 'people:level': 2.5626283367556466, 'people:community': 2.5626283367556466, 'people:policy': 2.135523613963039, 'people:bank': 4.271047227926078, 'people:australian': 2.135523613963039, 'people:debt': 2.5626283367556466, 'funeral_insurance:sunshine_coast': 4.139630390143737, 'funeral_insurance:people': 3.4496919917864477, 'funeral_insurance:indigenous_people': 4.829568788501026, 'funeral_insurance:death': 2.0698151950718686, 'funeral_insurance:funeral_plan': 3.2197125256673513, 'funeral_insurance:community_benefit': 3.2197125256673513, 'funeral_insurance:community': 3.2197125256673513, 'funeral_insurance:policy': 6.209445585215605, 'funeral_insurance:bank': 2.0698151950718686, 'funeral_insurance:see': 2.299794661190965, 'funeral_insurance:due': 2.9897330595482545, 'funeral_insurance:sydney': 3.2197125256673513, 'funeral_insurance:incentive_schemes': 3.4496919917864477, 'funeral_insurance:aboriginal': 4.369609856262834, 'funeral_insurance:misconduct': 

In [21]:
# Make CSV of bigram co-occurrence net
sourceNode=[]
targetNode=[]
edgeWeight=[]

for key,val in edge_weight_conet.items():
    composite_topics=key.split(':')
    sourceNode.append(composite_topics[0])
    targetNode.append(composite_topics[1])
    edgeWeight.append(val)
    
for key,val in edge_weight_direct.items():
    composite_topics=key.split(':')
    sourceNode.append(composite_topics[0])
    targetNode.append(composite_topics[1])
    edgeWeight.append(val)
    
cooccur_df=pd.DataFrame()
cooccur_df['source']=sourceNode
cooccur_df['target']=targetNode
cooccur_df['weight']=edgeWeight

cooccur_df.to_csv('30_topics_compositeEdge_bigram_cooccurrence_net.csv',index=False,encoding='utf-8')

In [22]:
# Make trigrams based on Inverse Frequency Distribution :

#edge_weight_conet=dict()
trigram_weight_conet=dict()
stop_words_trigram=['hear', 'business','australian','time','believe','support','public','new','fall','smh','html','make','june','child','year']

for key,val in edge_weight_conet.items():
    composite_topics=key.split(':')
    bi_source_topic=composite_topics[0]    
    bi_target_topic=composite_topics[1]
    if((bi_source_topic not in stop_words_trigram) and (bi_target_topic not in stop_words_trigram)):
        for key2,val2 in edge_weight_conet.items():
            composite_topics2=key2.split(':')
            inner_source_topic=composite_topics2[0]
            inner_target_topic=composite_topics2[1]
            #print(bi_target_topic , inner_source_topic)
            if (bi_target_topic == inner_source_topic and inner_target_topic != bi_source_topic and (inner_target_topic not in stop_words_trigram)):
                trigram_key=bi_source_topic+':'+bi_target_topic+':'+inner_target_topic
                trigram_weight=val*val2
                if(trigram_weight > .1):
                    trigram_weight_conet[trigram_key]=trigram_weight
            #if (bi_source_topic == inner_target_topic and inner_source_topic != bi_target_topic and (inner_source_topic not in stop_words_trigram)):
                #trigram_key2=inner_source_topic+':'+inner_target_topic+':'+bi_target_topic
                #trigram_weight2=val*val2
                #if(trigram_weight > .1):
                    #trigram_weight_conet[trigram_key2]=trigram_weight2
    
print(trigram_weight_conet)

{'sunshine_coast:incentive_schemes:sydney': 0.3, 'vulnerable_people:funeral:people': 0.15625, 'vulnerable_people:funeral:funeral_insurance': 0.171875, 'vulnerable_people:funeral:community': 0.21875, 'vulnerable_people:funeral:policy': 0.171875, 'vulnerable_people:funeral:aboriginal': 0.203125, 'vulnerable_people:funeral_insurance:policy': 0.12053571428571429, 'vulnerable_people:indigenous_community:people': 0.25, 'vulnerable_people:indigenous_community:funeral_insurance': 0.25, 'vulnerable_people:indigenous_community:funeral_plan': 0.25, 'vulnerable_people:indigenous_community:policy': 0.25, 'vulnerable_people:indigenous_community:see': 0.25, 'vulnerable_people:indigenous_community:majority': 0.25, 'vulnerable_people:community_benefit:funeral_plan': 0.1372549019607843, 'vulnerable_people:aboriginal_organisation:funeral_insurance': 0.16666666666666666, 'vulnerable_people:aboriginal_organisation:funeral_plan': 0.1111111111111111, 'vulnerable_people:aboriginal_organisation:community_benef

In [23]:
# Make trigrams based on Direct Frequency Distribution :

trigram_weight_direct=dict()
#stop_words_trigram=['hear', 'business','australian','time','believe','support','public','new','fall','smh','html','make','june','child','year']

for key,val in edge_weight_direct.items():
    composite_topics=key.split(':')
    bi_source_topic=composite_topics[0]    
    bi_target_topic=composite_topics[1]
    if((bi_source_topic not in stop_words_trigram) and (bi_target_topic not in stop_words_trigram)):
        for key2,val2 in edge_weight_direct.items():
            composite_topics2=key2.split(':')
            inner_source_topic=composite_topics2[0]
            inner_target_topic=composite_topics2[1]
            #print(bi_target_topic , inner_source_topic)
            if (bi_target_topic == inner_source_topic and inner_target_topic != bi_source_topic and (inner_target_topic not in stop_words_trigram)):
                trigram_key=bi_source_topic+':'+bi_target_topic+':'+inner_target_topic
                trigram_weight=val*val2
                if(trigram_weight > 60):
                    trigram_weight_direct[trigram_key]=trigram_weight            
    
print(trigram_weight_direct)

{'people:bank:group': 81.64979402873057, 'people:bank:level': 60.163006126433054, 'people:bank:government': 60.163006126433054, 'people:bank:home_loan': 81.64979402873057, 'people:bank:interest': 111.7312970919471, 'people:bank:policy': 85.94715160919009, 'people:bank:back': 77.35243644827106, 'people:bank:rural_bank': 64.46036370689255, 'people:bank:see': 94.54186677010908, 'people:bank:process': 77.35243644827106, 'people:bank:wealth_management': 103.13658193102809, 'people:bank:misconduct': 163.29958805746114, 'people:bank:debt': 111.7312970919471, 'funeral_insurance:bank:misconduct': 79.1374926740004, 'government:bank:people': 79.65813407317144, 'government:bank:interest': 73.9682673536592, 'government:bank:see': 62.58853391463471, 'government:bank:wealth_management': 68.27840063414696, 'government:bank:misconduct': 108.10746767073269, 'government:bank:debt': 73.9682673536592, 'policy:bank:people': 60.74149656995644, 'policy:bank:misconduct': 82.43488820208374, 'bank:people:level':

In [24]:
# Make CSV of trigram co-occurrence net
sourceNode_trig=[]
targetNode_trig=[]
edgeWeight_trig=[]

for key,val in trigram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]
    sourceNode_trig.append(source)
    targetNode_trig.append(composite_topics[2])
    edgeWeight_trig.append(val)
    
for key,val in trigram_weight_direct.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]
    sourceNode_trig.append(source)
    targetNode_trig.append(composite_topics[2])
    edgeWeight_trig.append(val)
    
tri_cooccur_df=pd.DataFrame()
tri_cooccur_df['source']=sourceNode_trig
tri_cooccur_df['target']=targetNode_trig
tri_cooccur_df['weight']=edgeWeight_trig

#tri_cooccur_df.to_csv('50top_lawDom_3gram__cooccurrence_net.csv',index=False,encoding='utf-8')
tri_cooccur_df.to_csv('30top_SLG_Dom_3gram_compositeEdge_cooccurrence_net.csv',index=False,encoding='utf-8')

In [25]:
# Make 4grams based on Inverse Frequency Distribution :

fourgram_weight_conet=dict()

for key3,val3 in trigram_weight_conet.items():
    composite_topics=key3.split(':')
    tri_source_topic1=composite_topics[0]
    tri_source_topic2=composite_topics[1]
    tri_target_topic=composite_topics[2]
    for key4,val4 in edge_weight_conet.items():
        composite_topics2=key4.split(':')
        inner_source_topic=composite_topics2[0]
        inner_target_topic=composite_topics2[1]
        #print(bi_target_topic , inner_source_topic)
        if (tri_target_topic == inner_source_topic and inner_target_topic != tri_source_topic1 and inner_target_topic != tri_source_topic2 and (inner_target_topic not in stop_words_trigram)):
            fourgram_key=tri_source_topic1+':'+tri_source_topic2+':'+tri_target_topic+':'+inner_target_topic
            fourgram_weight=val3*val4
            if(fourgram_weight > 0.1):
                fourgram_weight_conet[fourgram_key]=fourgram_weight
        #if (tri_source_topic1 == inner_target_topic and inner_source_topic != tri_target_topic and inner_source_topic != tri_source_topic2 and (inner_source_topic not in stop_words_trigram)):
            #fourgram_key=inner_source_topic+':'+inner_target_topic+':'+tri_source_topic2+':'+tri_target_topic
            #fourgram_weight=val3*val4
            #if(fourgram_weight > 0.1):
            #fourgram_weight_conet[fourgram_key]=fourgram_weight    
    
print(fourgram_weight_conet)

{'vulnerable_people:indigenous_community:funeral_plan:see': 0.125, 'funeral_insurer:aboriginal_organisation:scrutiny_aboriginal:funeral_plan': 0.10714285714285714, 'funeral_insurer:aboriginal_organisation:scrutiny_aboriginal:community_benefit': 0.10714285714285714, 'funeral_insurer:scrutiny_aboriginal:funeral_plan:see': 0.17857142857142858, 'funeral_insurer:scrutiny_aboriginal:community_benefit:funeral_plan': 0.14705882352941177, 'funeral_insurer:indigenous_australian:scrutiny_aboriginal:funeral_plan': 0.10714285714285714, 'funeral_insurer:indigenous_australian:scrutiny_aboriginal:community_benefit': 0.10714285714285714, 'sydney_call:sunshine_coast:incentive_schemes:sydney': 0.3, 'sydney_call:incentive_schemes:sunshine_coast:funeral_insurance': 0.18, 'sydney_call:incentive_schemes:sunshine_coast:sydney': 0.27, 'sydney_call:incentive_schemes:sydney:sunshine_coast': 0.2370731707317073, 'aboriginal_person:death:probe:due': 0.11882716049382716, 'aboriginal_person:indigenous_australian:abor

In [30]:
# Make 4grams based on Direct Frequency Distribution :

fourgram_weight_direct=dict()

for key3,val3 in trigram_weight_direct.items():
    composite_topics=key3.split(':')
    tri_source_topic1=composite_topics[0]
    tri_source_topic2=composite_topics[1]
    tri_target_topic=composite_topics[2]
    for key4,val4 in edge_weight_direct.items():
        composite_topics2=key4.split(':')
        inner_source_topic=composite_topics2[0]
        inner_target_topic=composite_topics2[1]
        #print(bi_target_topic , inner_source_topic)
        if (tri_target_topic == inner_source_topic and inner_target_topic != tri_source_topic1 and inner_target_topic != tri_source_topic2 and (inner_target_topic not in stop_words_trigram)):
            fourgram_key=tri_source_topic1+':'+tri_source_topic2+':'+tri_target_topic+':'+inner_target_topic
            fourgram_weight=val3*val4
            if(fourgram_weight > 200):
                fourgram_weight_direct[fourgram_key]=fourgram_weight        
    
print(fourgram_weight_direct)

{'people:bank:policy:member': 240.8991005062515, 'people:bank:policy:indigenous_people': 259.42980054519387, 'people:bank:policy:funeral_insurance': 352.083300739906, 'people:bank:policy:community': 203.83770042836665, 'people:bank:policy:aboriginal': 222.36840046730904, 'people:bank:misconduct:aboriginal': 407.41067657046256, 'government:bank:people:level': 204.1341916289896, 'government:bank:people:community': 204.1341916289896, 'government:bank:people:debt': 204.1341916289896, 'government:bank:misconduct:aboriginal': 269.71370271034954, 'policy:bank:misconduct:aboriginal': 205.6640434610508, 'bank:people:policy:funeral_insurance': 246.4583105179342, 'bank:policy:funeral_insurance:sunshine_coast': 341.2499684094473, 'bank:policy:funeral_insurance:people': 284.37497367453943, 'bank:policy:funeral_insurance:indigenous_people': 398.1249631443552, 'bank:policy:funeral_insurance:funeral_plan': 265.41664209623684, 'bank:policy:funeral_insurance:community_benefit': 265.41664209623684, 'bank

In [31]:
# Make CSV of fourgram co-occurrence net

sourceNode_4g=[]
targetNode_4g=[]
edgeWeight_4g=[]

for key,val in fourgram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]+':'+composite_topics[2]
    sourceNode_4g.append(source)
    targetNode_4g.append(composite_topics[3])
    edgeWeight_4g.append(val)

for key,val in fourgram_weight_direct.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]+':'+composite_topics[2]
    sourceNode_4g.append(source)
    targetNode_4g.append(composite_topics[3])
    edgeWeight_4g.append(val)
    
four_cooccur_df=pd.DataFrame()
four_cooccur_df['source']=sourceNode_4g
four_cooccur_df['target']=targetNode_4g
four_cooccur_df['weight']=edgeWeight_4g

four_cooccur_df.to_csv('30top_SLG_Dom_compositeEdge_4gram_cooccurrence_net.csv',index=False,encoding='utf-8')

In [32]:
# Try 5 gram based on Inverse Frequency Distribution :

#fourgram_weight_conet=dict()
fivegram_weight_conet=dict()

for key5,val5 in fourgram_weight_conet.items():
    composite_topics=key5.split(':')
    four_source_topic1=composite_topics[0]
    four_source_topic2=composite_topics[1]
    four_source_topic3=composite_topics[2]
    four_target_topic=composite_topics[3]
    for key6,val6 in edge_weight_conet.items():
        composite_topics2=key6.split(':')
        inner_source_topic=composite_topics2[0]
        inner_target_topic=composite_topics2[1]
        #print(bi_target_topic , inner_source_topic)
        if (four_target_topic == inner_source_topic and inner_target_topic != four_source_topic1 
            and inner_target_topic != four_source_topic2 and inner_target_topic != four_source_topic3
            and (inner_target_topic not in stop_words_trigram)):
            fivegram_key=four_source_topic1+':'+four_source_topic2+':'+four_source_topic3+':'+four_target_topic+':'+inner_target_topic
            fivegram_weight=val5*val6
            #if(fivegram_weight > 0.1):
            fivegram_weight_conet[fivegram_key]=fivegram_weight
    
    
print(fivegram_weight_conet)

{'funeral_insurer:aboriginal_organisation:scrutiny_aboriginal:funeral_plan:funeral_insurance': 0.026785714285714284, 'funeral_insurer:aboriginal_organisation:scrutiny_aboriginal:funeral_plan:see': 0.05357142857142857, 'funeral_insurer:aboriginal_organisation:scrutiny_aboriginal:community_benefit:funeral_plan': 0.044117647058823525, 'funeral_insurer:aboriginal_organisation:scrutiny_aboriginal:community_benefit:bank': 0.025210084033613443, 'funeral_insurer:aboriginal_organisation:scrutiny_aboriginal:community_benefit:aboriginal': 0.025210084033613443, 'funeral_insurer:scrutiny_aboriginal:community_benefit:funeral_plan:funeral_insurance': 0.03676470588235294, 'funeral_insurer:scrutiny_aboriginal:community_benefit:funeral_plan:see': 0.07352941176470588, 'funeral_insurer:indigenous_australian:scrutiny_aboriginal:funeral_plan:funeral_insurance': 0.026785714285714284, 'funeral_insurer:indigenous_australian:scrutiny_aboriginal:funeral_plan:see': 0.05357142857142857, 'funeral_insurer:indigenous

In [67]:
# Make CSV of fivegram co-occurrence net

sourceNode_5g=[]
targetNode_5g=[]
edgeWeight_5g=[]

for key,val in fivegram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]+':'+composite_topics[2]+':'+composite_topics[3]
    sourceNode_5g.append(source)
    targetNode_5g.append(composite_topics[4])
    edgeWeight_5g.append(val)
    
five_cooccur_df=pd.DataFrame()
five_cooccur_df['source']=sourceNode_5g
five_cooccur_df['target']=targetNode_5g
five_cooccur_df['weight']=edgeWeight_5g

five_cooccur_df.to_csv('50_topics_5gram_ooccurrence_net.csv',index=False,encoding='utf-8')